In [ ]:
# parameters

In [ ]:
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
Mycelia.NERSC_CPU

In [ ]:
mem_gb = Int(floor(Mycelia.NERSC_MEM * .9))

In [ ]:
cores = Mycelia.NERSC_CPU

In [ ]:
# minimap2 [-x preset] -d target.mmi target.fa
# minimap2 -a [-x preset] target.mmi query.fa > output.sam

In [ ]:
function minimap_pacbio_index(;
        reference_fasta,
        outfile = reference_fasta * "",
        threads = 4,
        memory = Sys.total_memory(),
        shell_only = false
    )
    # 4G is the default
    # smaller, higher diversity databases do better with 5+ as the denominator - w/ <=4 they run out of memory
    index_chunk_size = "$(Int(floor(memory/5e9)))G"
    @show index_chunk_size
    @show threads
    Mycelia.add_bioconda_env("minimap2")
    outfile = "$(reference_fasta).I$(index_chunk_size).x_map-hifi.mmi"
    if shell_only
        cmd =
        """
        $(Mycelia.CONDA_RUNNER) run --live-stream -n minimap2 minimap2 -t $(threads) -I$(index_chunk_size) -x map-hifi $(reference_fasta) -d $(outfile)
        """
        return cmd
    else
        if !isfile(outfile)
            cmd = `$(Mycelia.CONDA_RUNNER) run --live-stream -n minimap2 minimap2 -t $(threads) -I$(index_chunk_size) -x map-hifi $(reference_fasta) -d $(outfile)`
            run(cmd)
            @assert isfile(outfile)
        else
            @info "$(outfile) already present"
        end
    end
end

# sample 1 - prokaryotic only
cmd = minimap_pacbio_index(
    reference_fasta = reference_fasta,
    shell_only = true,
    threads = cores,
    memory = mem_gb * 1e9
)

In [ ]:
job_name = "$(basename(reference_fasta)).mmi"
println(job_name)

Mycelia.nersc_sbatch_premium(
    job_name=job_name,
    mail_user="cameron.prybol@gmail.com",
    logdir=mkpath("$(homedir())/workspace/slurmlogs"),
    mem_gb=mem_gb,
    cpus_per_task=cores,
    cmd=cmd)

In [ ]:
# reference_fasta = "$(homedir())/workspace/blastdb/nt_prok.fna.gz"
reference_fasta = "$(homedir())/workspace/blastdb/nt.fna.gz"

# fastq = "/global/homes/c/cjprybol/workspace/Mycelia/projects/metagenome/data/SRR9202034/SRR9202034.seqkit.P0.0001.fq.gz"
# fastq = "/global/homes/c/cjprybol/workspace/Mycelia/projects/metagenome/data/SRR9202034/SRR9202034.seqkit.P0.001.fq.gz"
# fastq = "/global/homes/c/cjprybol/workspace/Mycelia/projects/metagenome/data/SRR9202034/SRR9202034.seqkit.P0.01.fq.gz"
# fastq = "/global/homes/c/cjprybol/workspace/Mycelia/projects/metagenome/data/SRR9202034/SRR9202034.seqkit.P0.1.fq.gz"
fastq = "/global/homes/c/cjprybol/workspace/Mycelia/projects/metagenome/data/SRR9202034/SRR9202034.fastq.gz"

# sample 1 - prokaryotic only
cmd = Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = reference_fasta,
    shell_only = true,
    threads = cores,
    memory = mem_gb * 1e9
)



job_name = basename(fastq) * "_" * basename(reference_fasta)
print(job_name)
# Mycelia.nersc_sbatch(
#     job_name=job_name,
#     mail_user="cameron.prybol@gmail.com",
#     cpus_per_task=cores,
#     mem_gb=mem_gb,
#     qos="premium",
#     logdir=mkpath("$(homedir())/workspace/slurmlogs"),
#     cmd=cmd)

Mycelia.nersc_sbatch_regular(
    job_name=job_name,
    mail_user="cameron.prybol@gmail.com",
    logdir=mkpath("$(homedir())/workspace/slurmlogs"),
    mem_gb=mem_gb,
    cpus_per_task=cores,
    cmd=cmd)

In [ ]:
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = "$(homedir())/workspace/blastdb/nt_viruses.fna.gz",
)

In [ ]:
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = "$(homedir())/workspace/blastdb/nt_others.fna.gz",
)

In [ ]:
# sample 1 - protozoa
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = "$(homedir())/workspace/blastdb/protozoa.fna.gz",
)

In [ ]:
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = "$(homedir())/workspace/blastdb/fungi.fna.gz",
)

In [ ]:
# sample 1 - plant only
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = "$(homedir())/workspace/blastdb/plant.fna.gz",
)

In [ ]:
# sample 1 - invertebrate only
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    # reference_fasta = "$(homedir())/workspace/blastdb/invertebrate.fna.gz",
    reference_fasta = "$(homedir())/workspace/blastdb/invertebrate.deduped.fna.gz"
)

In [ ]:
# sample 1 - vertebrate_other
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = "$(homedir())/workspace/blastdb/vertebrate_other.fna.gz",
)

In [ ]:
# sample 1 - vertebrate_mammal
@time Mycelia.map_pacbio_reads(
    fastq = fastq,
    reference_fasta = "$(homedir())/workspace/blastdb/vertebrate_mammalian.fna.gz",
)

In [ ]:
## QUESTION 1 - ARE ALIGNMENTS & MAPQ STABLE OR DO THEY CHANGE WITH DATABASE SIZE?